#### 사례
"나는 김치찌개를 만들어 먹었다."라는 문장이 있을 때, 우리가 "만들어"라는 단어에 주목하려 한다고 가정. 여기서 "만들어"는 쿼리(Query). 이제 문장의 모든 단어들이 키(Key)와 값(Value)를 갖는다.

"나는": 키(Key)로서 "나는"이라는 주체를 나타내고, 값(Value)으로서는 "나는"라는 개체에 대한 정보를 갖는다.

"김치찌개를": 키(Key)로서 "김치찌개를"이라는 대상을 나타내고, 값(Value)으로서는 "김치찌개"라는 대상에 대한 정보를 갖는다.

"만들어": 키(Key)로서 "만들어"라는 행동을 나타내고, 값(Value)으로서는 "만들어"라는 행동에 대한 정보를 갖는다.

"먹었다.": 키(Key)로서 "먹었다."라는 행동을 나타내고, 값(Value)으로서는 "먹었다"라는 행동에 대한 정보를 갖는다.

- 입력 준비: 문장의 각 단어에 대해 세 가지 다른 가중치 행렬을 곱하여 쿼리(Query), 키(Key), 값(Value) 벡터를 생성합니다. 이 가중치 행렬들은 모델의 학습 과정에서 최적화됩니다.

- 스코어 계산: "만들어"라는 쿼리 벡터와 각 키 벡터의 내적을 계산하여 스코어를 계산합니다. 이 스코어는 "만들어"라는 단어가 각 단어("나는", "김치찌개를", "만들어", "먹었다.")와 얼마나 관련이 있는지를 나타냅니다.

- 스코어 정규화: 계산된 스코어를 softmax 함수를 통해 정규화하여 확률분포를 만듭니다. 이 확률분포는 "만들어"라는 단어가 각 단어에 얼마나 가중치를 둘 것인지를 나타냅니다.

- 가중합 계산: 정규화된 스코어를 각 값 벡터에 곱한 후, 모두 합하여 새로운 벡터를 생성합니다. 예를 들어, "김치찌개를"에 대한 스코어가 높다면, "김치찌개"에 대한 값 벡터가 "만들어"라는 단어의 새로운 표현에 더 크게 기여하게 됩니다.

- 출력 생성: 모든 입력 원소에 대해 위의 과정을 반복하고, 그 결과를 합쳐서 셀프 어텐션 레이어의 최종 출력을 생성합니다. 이렇게 생성된 출력은 다음 레이어로 전달됩니다.

In [1]:
import torch

x = torch.tensor([
    [1.0, 0.0, 1.0, 0.0],
    [0.0, 2.0, 0.0, 2.0],
    [1.0, 1.0, 1.0, 1.0],
])

w_query = torch.tensor([
    [1.0, 0.0, 1.0],
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0]
])
w_key = torch.tensor([
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0]
])
w_value = torch.tensor([
    [0.0, 2.0, 0.0],
    [0.0, 3.0, 0.0],
    [1.0, 0.0, 3.0],
    [1.0, 1.0, 0.0]
])

In [2]:
keys = torch.matmul(x, w_key)
querys = torch.matmul(x, w_query)
value = torch.matmul(x, w_value)

In [3]:
print(keys, '\n')
print(querys, '\n')
print(value, )

tensor([[0., 1., 1.],
        [4., 4., 0.],
        [2., 3., 1.]]) 

tensor([[1., 0., 2.],
        [2., 2., 2.],
        [2., 1., 3.]]) 

tensor([[1., 2., 3.],
        [2., 8., 0.],
        [2., 6., 3.]])


In [4]:
attn_scores = torch.matmul(querys, keys.T)

In [5]:
attn_scores

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

In [7]:
import numpy as np

print(keys.shape)
np.sqrt(keys.shape[-1])

torch.Size([3, 3])


1.7320508075688772

In [8]:
# 소프트 맥스 확률값 만들기
# 키 벡터의 차원수의 제곱근을 스케일링 요소로 사용, 키의 차원이 클때 attn_scores가 너무 커지는 것을 방지
# 각 쿼리에 대해 모든 키에 대한 스코어들이 합쳐져 1이 되도록함

from torch.nn.functional import softmax

key_dim_sqrt = np.sqrt(keys.shape[-1])
attn_scores_softmax = softmax(attn_scores / key_dim_sqrt, dim = -1)

In [9]:
attn_scores_softmax

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

In [10]:
attn_scores_softmax[1].sum()

tensor(1.0000)

가중합 계산

In [12]:
weighted_values = torch.matmul(attn_scores_softmax, value)

In [14]:
# 소프트맥스 확률과 밸류를 가중합하기
# 정규화된 어텐션 스코어 를 값 행열에 곱하여 가중합을 계산
# 이 표현은 각 키와 연관된 값의 가중합으로 구성, 이렇게 생성된 새로운 표현은 문장의 다른부분에서 가져온 정보를 포함하므로
# 쿼리 단어의 문맥을 반영한 표현

weighted_values

tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])

출력 생성

In [21]:
x = torch.tensor([2, 1])
w1 = torch.tensor([[3,2,-4],[2, -3, 1]])
b1 = 1
w2 = torch.tensor([[-1,1],[1,2], [3, 1]])
b2 = -1

In [22]:
print(x, '\n')
print(w1, '\n')
print(w2, '\n')

tensor([2, 1]) 

tensor([[ 3,  2, -4],
        [ 2, -3,  1]]) 

tensor([[-1,  1],
        [ 1,  2],
        [ 3,  1]]) 



In [19]:
h_preact = torch.matmul(x, w1) + b1
h = torch.nn.functional.relu(h_preact)
y = torch.matmul(h, w2) + b2

In [20]:
print(h_preact, '\n')
print(h, '\n')
print(y)

tensor([ 9,  2, -6]) 

tensor([9, 2, 0]) 

tensor([-8, 12])


### 드롭아웃
- 딥러닝 모델은 그 표현력이 아주 좋아서 학습 데이터 그 자체를 외워버릴 염려가 있으며 이를 과적합(overfitting)이라고 한다.
- 드롭아웃(dropout)은 이러한 과적합 현상을 방지하고자 뉴런의 일부를 확률적으로 0으로 대치하여 계산에서 제외하는 기법

In [24]:
m = torch.nn.Dropout(p = 0.2)
input = torch.randn(1, 10)
output = m(input)

In [25]:
input

tensor([[ 0.5630, -0.4729,  0.1320, -1.5207,  0.2673,  0.4819,  0.4714, -0.5229,
          0.2142,  0.2838]])

In [26]:
output

tensor([[ 0.7038, -0.5912,  0.1650, -1.9009,  0.0000,  0.6024,  0.0000, -0.6536,
          0.2677,  0.0000]])

### 아담 옵티마이저
- 딥러닝 모델 학습은 모델 출력과 정답 사이의 오차(error)를 최소화하는 방향을 구하고 이 방향에 맞춰 모델 전체의 파라미터(parameter)들을 업데이트하는 과정
- 오차를 손실(loss), 오차를 최소화하는 방향을 그래디언트(gradient)라고 합니다. 오차를 최소화하는 과정을 최적화(optimization)라고 한다.
- 오차를 구하려면 현재 시점의 모델에 입력을 넣어봐서 처음부터 끝까지 계산해보고 정답과 비교해야 하는데 오차를 구하기 위해 모델 처음부터 끝까지 순서대로 계산해보는 과정을 순전파(forward propagation)라고 한다.
- 오차를 구했다면 오차를 최소화하는 최초의 그래디언트를 구할 수 있다. 이는 미분(devative)으로 구하며 이후 미분의 연쇄 법칙(chain rule)에 따라 모델 각 가중치별 그래디언트 역시 구할 수 있으며 순전파의 역순으로 순차적으로 수행되서 이를 역전파(backpropagation)라고 한다.
- 아담 옵티마이저의 핵심 동작 원리는 방향과 보폭을 적절하게 정해주는 것이며 방향을 정할 때는 현재 위치에서 가장 경사가 급한 쪽으로 내려가되, 여태까지 내려오던 관성(방향)을 일부 유지하도록 한다. 보폭의 경우 안가본 곳은 성큼 빠르게 걸어 훑고 많이 가본 곳은 갈수록 보폭을 줄여 세밀하게 탐색하는 방식으로 정한다.

## BERT(Bidirectional Encoder Representations from Transformers)
- 트랜스포머 아키텍처의 인코더만을 사용하는데, 그 이유는 BERT의 주된 목표와 학습 방식 때문이다.
-  BERT는 문장의 일부 단어가 마스킹(가려짐)되어 있는 상황에서 마스킹된 단어를 예측하려고 하며 이 과정에서 BERT는 양방향의 문맥 정보를 모두 활용하여 단어의 의미를 이해하게 된다.
- 트랜스포머의 인코더는 입력 시퀀스 내의 모든 위치에서 모든 위치로의 어텐션을 계산하므로, 각 단어에 대한 양방향 문맥을 자연스럽게 학습하므로 이러한 이유로 BERT는 트랜스포머의 인코더만을 사용
- 디코더는 대부분의 경우에 순차적인 정보 처리를 요구하는데, 이는 BERT의 양방향 문맥 이해 목표와는 부합하지 않기 때문

#### bert의 토크나이저 방법
- WordPiece 토크나이징 방식은 원래 문장을 더 작은 단위로 나누는 방법. 이 방식은 처음에는 각 문자를 개별 토큰으로 간주하고, 빈도 기반의 방법을 사용하여 반복적으로 가장 빈도가 높은 바이그램(두 문자의 연속)을 하나의 토큰으로 합치며 토큰 집합의 크기가 사전에 정한 어휘 크기에 도달할 때까지 반복.
-  예를 들어, 'unhappiness'라는 단어는 'un', '##happy', '##ness'라는 세 개의 토큰으로 분리될 수 있으며 '##' 기호는 해당 토큰이 원래 단어의 첫 토큰이 아니라는 것을 나타낸다.
- 이렇게 하면 BERT는 훈련 데이터에 없는 단어에 대해서도 일반화를 할 수 있게 되며, 이는 다양한 언어와 도메인에 대해 효과적으로 작동.

bert의 학습방법
사전 학습(Pre-training): 이 단계에서는 큰 텍스트 코퍼스(예: Wikipedia)를 사용하여 모델을 학습. BERT의 사전 학습은 두 가지 비지도 학습 방법을 사용.

Masked Language Model(MLM): 일부 단어를 마스킹하고, 마스킹된 단어를 예측하도록 모델을 학습. 이를 통해 모델은 문맥을 이해하고, 주변 단어를 기반으로 단어를 예측하는 능력을 키울 수 있다.

Next Sentence Prediction(NSP): 두 문장이 주어졌을 때, 두 번째 문장이 첫 번째 문장 다음에 오는 문장인지를 예측하도록 모델을 학습. 이를 통해 모델은 문장 간의 관계를 이해하는 능력을 키울 수 있다.

파인 튜닝(Fine-tuning): 사전 학습 후, BERT 모델은 특정 작업에 대해 파인 튜닝 될 수 있다. 파인 튜닝은 레이블이 달린 작은 데이터셋을 사용하여 진행되며, 모든 레이어의 가중치를 업데이트. 이 단계에서는 특정 작업(예: 감성 분석, 질문 응답 등)의 학습 데이터를 사용하여 모델을 학습.

이런 방식으로 BERT는 먼저 비지도 학습 방법을 통해 언어의 일반적인 패턴을 학습하고, 그 다음에는 지도 학습 방법을 통해 특정 작업에 맞게 모델을 조정. 이는 BERT가 다양한 자연어 처리 작업에 효과적으로 적용될 수 있게 만들어 준다.

## GPT (Generative Pretrained Transformer)
- 트랜스포머 아키텍처의 디코더만 사용하는데, 그 이유는 GPT의 목표와 학습 방식에 기인.
- GPT는 주어진 문맥에 따라 텍스트를 생성하는 것을 목표로 하며 주어진 입력에 대해 가장 가능성 있는 다음 토큰(일반적으로 단어 또는 단어의 일부)을 예측하는 문제로 볼 수 있다.
- 입력 텍스트를 디코더에 직접 제공하고, 디코더는 각 단계에서 다음 토큰을 예측하게 되어 GPT는 주어진 문맥을 기반으로 텍스트를 생성하는 데 필요한 모델을 학습

#### GPT 토큰화 방법
- GPT(Generative Pretrained Transformer)는 토큰화를 위해 Byte-Pair Encoding(BPE)라는 방법을 사용
- BPE는 원래의 텍스트에서 가장 빈번하게 등장하는 문자열 조합을 찾아서 하나의 새로운 '단어'를 만드는 과정을 반복적으로 수행하 자주 등장하는 단어는 하나의 토큰으로 묶이고, 그렇지 않은 단어는 여러 개의 토큰으로 분리
- 예를 들어 'lowest'라는 단어가 자주 등장하지 않고, 'low'라는 단어와 'est'라는 문자열이 자주 등장한다면, BPE는 'lowest'를 'low'와 'est' 두 개의 토큰으로 분리.
- 이런 방식으로 BPE는 텍스트를 더 작은 단위로 토큰화할 수 있으며, 이는 GPT와 같은 트랜스포머 기반 모델이 문맥을 보다 정확하게 이해하고, 더 큰 어휘를 처리

#### GPT 학습방법
- 사전 학습 (Pre-training): 이 단계에서 GPT는 대량의 텍스트 데이터를 학습하여 언어의 통계적 패턴을 이해. GPT는 이를 위해 'masked language model'이 아닌 'causal language model'을 사용. 즉, 주어진 문장에서 다음에 올 단어를 예측하는 방식으로 학습. 이 때문에 GPT는 문장을 앞에서부터 뒤로 읽으며 학습하게 되므로, 이를 'auto-regressive' 방식이라고 함.

- 파인 튜닝 (Fine-tuning): 사전 학습 후, GPT는 특정 작업에 대해 파인 튜닝될 수 있다. 이 단계에서는 특정 작업(예: 감성 분석, 질문 응답 등)의 학습 데이터를 사용하여 모델을 학습시킵니다. 이때, 모델의 모든 가중치는 업데이트 가능하며, 파인 튜닝을 통해 특정 작업에 대해 최적화됩니다.

- 따라서, GPT의 학습 방법은 사전 학습과 파인 튜닝 두 가지 주요 단계로 구성됩니다. 사전 학습 단계에서는 언어의 일반적인 패턴을 학습하고, 파인 튜닝 단계에서는 이를 바탕으로 특정 작업에 대해 최적화를 진행한다.

- GPT의 원래 목적은 주어진 문맥에서 다음 단어를 예측하는 것이지만, 이런 방식은 기본적으로 언어의 구조와 문맥을 이해하는 능력을 요구. 따라서 이를 바탕으로 모델을 약간 수정하면, 문맥을 이해하고 특정 작업을 수행할 수 있는 모델을 만들 수 있다.
  - 예를 들어, 감성 분석 작업을 수행하기 위해 GPT를 파인 튜닝하는 경우, 모델의 출력 레이어를 각각의 감성 클래스에 대응하는 노드로 대체한 다음, 감성 레이블이 부여된 데이터셋을 사용하여 모델을 학습시키면, GPT는 입력 텍스트의 감성을 분류하는 데 필요한 패턴을 학습하게 된다.

  - 마찬가지로, 질문 응답 시스템을 구축하기 위해 GPT를 파인 튜닝하는 경우, 모델의 입력을 질문과 문맥, 그리고 가능한 답변의 시작과 끝을 나타내는 토큰으로 구성한 다음, 질문과 그에 대한 정답이 포함된 데이터셋을 사용하여 모델을 학습시키면, GPT는 주어진 문맥에서 질문에 대한 적절한 답변을 생성하는 능력을 학습하게 된다.

  따라서, GPT는 언어 생성 모델이지만, 파인 튜닝을 통해 다양한 자연어 처리 작업에 적용할 수 있다. 이는 GPT가 언어의 복잡한 패턴과 구조를 이해하는 능력을 가지고 있기 때문에 가능한 것이다.

## 파인튜닝

- 문장을 워드피스(wordpiece)로 토큰화한 뒤 앞뒤에 문장 시작과 끝을 알리는 스페셜 토큰 CLS와 SEP를 각각 추가한 뒤 BERT에 입력

- BERT 모델의 마지막 블록(레이어)의 출력 가운데 CLS에 해당하는 벡터를 추출. 트랜스포머 인코더 블록에서는 모든 단어가 서로 영향을 끼치기 때문에 마지막 블록 CLS 벡터는 문장 전체(이 영화 재미없네요)의 의미가 벡터 하나로 응집된 것이다.

- 이렇게 뽑은 CLS 벡터에 작은 모듈을 하나 추가해, 그 출력이 미리 정해 놓은 범주(예컨대 긍정, 중립, 부정)가 될 확률이 되도록 한다. 학습 과정에서는 BERT와 그 위에 쌓은 작은 모듈을 포함한 전체 모델의 출력이 정답 레이블과 최대한 같아지도록 모델 전체를 업데이트. 이것이 파인튜닝(fine-tuning)

- 문서 분류는 마지막 블록의 CLS 벡터만을 사용하는 반면, 개체명 인식 같은 과제에서는 마지막 블록의 모든 단어 벡터를 활용